# nb2: The Coalescent and sequence variation

## The Coalescent 

This lesson will introduce Coalescent theory and the process by which coalescent trees can be randomly generated, and treated as stochastic random variables. For this, we will use the `ipcoal` library to perform stochastic coalescent simulations, and `toytree` to visualize trees.

In [ ]:
import toytree
import ipcoal

## Wright-Fisher models and coalescence

In the last lesson we introduced the Wright-Fisher (WF) process to simulate evolution in a discrete random mating population of size N diploid gene copies. Over multiple generations of random mating, genetic drift arises from random variation in which gene copies are replicated from one generation to the next. This has the consequence that genetic variation in the population is slowly reduced over time, as different alleles become lost or fixed. A second important outcome of this process is that it creates a network of ancestor-descendant relationships among gene copies (a genealogy). The loss/fixation of genetic variation over time, and the genealogical relationship among gene copies, are inherently linked. 

## Effective population size

One of the most challenging concepts in population genetics is that of effective population size ($N_e$). This is not to be confused with the population census size (N), which is the number of diploid individuals in a population.

The concept of effective population size was first developed by Sewall Wright (1931, 1933) as a metric for describing the rate of genetic drift, or inbreeding. Specifically, it was developed in reference to populations evolving under a WF process, where an idealized population will experience changes in allele frequencies over time due to only a single evolutionary force, genetic drift caused by a finite population size (N). It is *only* in the context of a WF process that N=$Ne$, by definition. 

In real data, the effective population is usually smaller than the population census size, representing the effects of many factors causing deviations from the assumptions of an idealized population. In the WF process model, we know that the finite population size N is the only factor causing deviations from an idealized population, and thus the two are equal.

In more complex models, $Ne$ is a summary statistic for the effects of many possible evolutionary processes that violate the assumptions of an idealized population, causing genetic drift and inbreeding at higher rates than would occur in a population of the same size if it were an idealized population. For example, a WF population of census size N=10,000 will have an effective population size of $N_e$=10,000. By contrast, a different population of census size N=10,000 that experiences non-random mating may have an effective population size of $N_e$=6,000. This value can be interpreted to mean that the second population will experience genetic drift at the same rate as an WF population of size N=6,000, even though it in fact contains 10,000 individuals.

### Coalescent simulation using *ipcoal*
Coalescent simulations provide a much faster approach to generating genealogies for a set of samples than the WF process. This is because in the WF process if we want to model a population of 10,000 individuals, we need to keep track of information for all 20,000 gene copies in the population. This is the case even if we only want to study the relationships among a small number of individuals sampled at the end of the simulation.

Coalescent simulation by contrast performs a process backwards in time, starting with the samples of interest at the present, and simulates only their ancestors. In this way, you do not need to simulate ancestry for the other ~2N gene copies in the population, nor do you need to simulate farther back in time than the most recent common ancestor of the samples you are studying. To do this, the Coalescent makes use the probabilistic expectations for when any two samples in a population of size N will coalesce. These 'waiting times' between coalescent events can be sampled as stochastic random variables and used to construct a genealogy. 

The coalescent simulation approach has become a standard tool in population genetics for generating null expectations for the genealogical relationships among a set of k samples in a population of size N. As we will see, the model can be extended to relax additional assumption of the WF process, such as by adding mutations, population structure (migration), and recombination, in addition to genetic drift.

## Single population coalescent

Let us start by simulating coalescent genealogies within a single population. Note, that just like in our WF process simulations, the relationships among gene copies in a single population are random at each simulated genealogy. This is because there is no population structure separating the samples, and within this population we have the assumption of random mating.

The N parameter in this model is equal to `Ne` unless we were to relax additional model assumptions. Here, again, similar to the WF process, when `Ne` is larger the time until coalescence into a common ancestor is also larger. This is demonstrated in two visualizations below, each showing four simulated coalescent trees for a population with small `Ne` or large `Ne`. 

### Low Ne
Here are simulating coalescent trees for a single population. For now, don't worry if you don't understand all details of the Python code, but do have a look to try to understand how it works. The comments explain what each step is doing. As a takeaway, note how the y-axis scale varies between the two plots below.

In [ ]:
import toyplot
toyplot.__version__

In [ ]:
# set up a single population simulation
model = ipcoal.Model(Ne=2_000, nsamples=10, seed_trees=123)

# generate 4 independent genealogical trees (i.e., 4 different genomic regions)
model.sim_trees(4)

# draw the 4 genealogies
model.draw_genealogies(shared_axes=True, scale_bar=True, node_sizes=0);

### High Ne
The genealogies at each locus are the same as before, but because the `Ne` value is 10X higher, the times between coalescent event are also 10X higher.

In [ ]:
# set up a single population simulation
model = ipcoal.Model(Ne=20_000, nsamples=10, seed_trees=123)

# generate 4 independent genealogical trees (i.e., 4 different genomic regions)
model.sim_trees(4)

# draw the 4 genealogies
model.draw_genealogies(shared_axes=True, scale_bar=True, node_sizes=0);

## Sequence variation
It is simple to also simulate sequences using coalescent simulations by simply modeling mutations as a process that occurs probabilistically along the edges of a genealogy. When we apply mutations into coalescent simulations the times between coalescent events (lengths of the genealogy branches) have important consequence, since they determine the opportunity for mutations to arise *within the history of the genealogy*. This is important, because the samples at the tips of the genealogy can only vary with respect to mutations that arise on their genealogical branches. If no mutations occur on the branches of this genealogy then all samples will have the same genotype that their common ancestor did. 

The mutation rate is typically described in units of mutations per site per generation. For example, 1e-8 is a typical value. Below, we measure the genetic variation among 10 samples in a population from the low versus high `Ne` scenarios above. As you can see, when `Ne` is greater, the coalescent branch lengths are longer, and the proportion of variable sites among the 10 sampled individuals is greater.

In [ ]:
model = ipcoal.Model(Ne=2_000, nsamples=10, mut=1e-8)
model.sim_loci(nloci=500, nsites=100)
print(f"% variant sites={model.df.nsnps.sum() / model.df.nbps.sum()} when Ne=2,000")

In [ ]:
model = ipcoal.Model(Ne=10_000, nsamples=10, mut=1e-8)
model.sim_loci(nloci=500, nsites=1000)
print(f"% variant sites={model.df.nsnps.sum() / model.df.nbps.sum()} when Ne=10,000")

## Summary
- Coalescent simulation is a much faster approach than forward-in-time simulations (e.g., WF) to simulate evolution.
- Larger Ne generates larger times between coalescent events
- Populations with larger Ne contain more genetic variation because the samples in this population are related by longer coalescent edge lengths on their genealogies.